In [8]:
import os
import time
import json
import openai
from dotenv import load_dotenv
from typing import List
from pydantic import BaseModel
from langgraph.checkpoints.sqlite import MemorySaver


load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")


memory = MemorySaver("chat_memory.sqlite")


class TravelPlan(BaseModel):
    destination: str
    date: str
    budget: str
    preferences: List[str]
    itinerary_suggestions: List[str]
    travel_tips: List[str]



ModuleNotFoundError: No module named 'langgraph.checkpoints'

In [ ]:
import os
import time
import json
import openai
from dotenv import load_dotenv
from typing import List
from pydantic import BaseModel
from langgraph.checkpoints.sqlite import MemorySaver

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

memory = MemorySaver("chat_memory.sqlite")

class TravelPlan(BaseModel):
    destination: str
    date: str
    budget: str
    preferences: List[str]
    itinerary_suggestions: List[str]
    travel_tips: List[str]


ModuleNotFoundError: No module named 'langgraph.checkpoints'

In [10]:
question_flow = [
    ("destination", "Hi! I'm here to help you plan your trip. Where would you like to go?"),
    ("date", "When do you plan to travel?"),
    ("budget", "What is your travel budget?"),
    ("preferences", "What activities do you enjoy? (e.g., museums, nature, food, nightlife)?")
]

def next_prompt(state):
    for key, prompt in question_flow:
        if not state.get(key):
            return key, prompt
    state["complete"] = True
    return None, "I think I have everything I need. Would you like me to generate your travel plan now?"

def trim_messages(history, limit=10):
    return history[-limit:]


In [ ]:
def stream_chat_completion(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.7,
        stream=True
    )
    full_reply = ""
    for chunk in response:
        delta = chunk['choices'][0]['delta']
        if "content" in delta:
            print(delta["content"], end='', flush=True)
            full_reply += delta["content"]
    print() 
    return full_reply



In [ ]:
def generate_plan(state):
    prompt = [
        {"role": "system", "content": "You are a helpful travel planner."},
        {"role": "user", "content": f"""
Based on this user's travel details:
Destination: {state['destination']}
Date: {state['date']}
Budget: {state['budget']}
Preferences: {', '.join(state['preferences'])}

Please return:
- 3 personalized itinerary suggestions (bullet list)
- 3 travel tips (bullet list)
"""}
    ]
    print("\nGenerating travel plan...\n")
    response = stream_chat_completion(prompt)

    itinerary = []
    tips = []
    lines = response.split("\n")
    current = itinerary

    for line in lines:
        line = line.strip("-• ").strip()
        if not line:
            continue
        if "tip" in line.lower():
            current = tips
        else:
            current.append(line)

    plan = TravelPlan(
        destination=state["destination"],
        date=state["date"],
        budget=state["budget"],
        preferences=state["preferences"],
        itinerary_suggestions=itinerary[:3],
        travel_tips=tips[:3] or ["Stay safe!", "Respect local customs", "Stick to your budget"]
    )

    print("\nFinal Travel Plan:")
    print(json.dumps(plan.dict(), indent=2))


In [ ]:
def run_chat():
    print("Welcome to the Travel Planning Chatbot!\n")

    # Restore from LangGraph memory or start new
    state = memory.get("user_session") or {
        "destination": None,
        "date": None,
        "budget": None,
        "preferences": [],
        "complete": False
    }

    message_history = []

    while not state["complete"]:
        key, prompt = next_prompt(state)
        print(prompt)

        user_input = input("> ").strip()

        message_history.append({"role": "assistant", "content": prompt})
        message_history.append({"role": "user", "content": user_input})
        message_history = trim_messages(message_history)

        if key == "preferences":
            state[key] = [x.strip() for x in user_input.split(",")]
        elif key:
            state[key] = user_input
        elif user_input.lower() in ["yes", "y"]:
            break
        else:
            print("Okay, let me know when you're ready.")
            continue

        memory.put("user_session", state)

    state["complete"] = True
    memory.put("user_session", state)
    generate_plan(state)


